# Architecture Hotspots

We are looking for hotspots on the project.

# Setting Up

In [1]:
import pandas as pd
import calendar

history = pd.read_json("../datasets/git_history_junit4.gz", encoding='utf-8-sig')
classes = pd.read_json("../datasets/class_structure_junit4.gz", encoding='utf-8-sig')

# To stop pandas from removing data
history = history.fillna("")
classes = classes.fillna("")

# Exploring Data

In [2]:
history.head()

,sha,date,time,author,author_email,author_id,commiter,message,modificationKind,file,fileType,class,package
0,3a5c6b4d08f408c8ca6a8e0bae71a9bc5a8f97e8,2020-08-02,20:24:18 +0200,Rahul Nagekar <rahul.nagekar1@gmail.com>,rahul.nagekar1@gmail.com,Rahul Nagekar <rahul.nagekar1@gmail.com>,GitHub <noreply@github.com>,Deprecated since jdk9 replacing constructor in...,M,src/main/java/junit/framework/Assert.java,java,Assert,/junit/framework
1,44cd8f80dbac11849a42a5d73fc9b5ddc75cd1b4,2016-12-01,21:35:45 +0100,Kevin Cooney <kcooney@google.com>,kcooney@users.noreply.github.com,Kevin Cooney <kcooney@users.noreply.github.com>,Stefan Birkner <github@stefan-birkner.de>,Change generics on ExpectedException.expectCau...,M,src/main/java/org/junit/internal/matchers/Thro...,java,ThrowableCauseMatcher,/org/junit/internal/matchers
2,44cd8f80dbac11849a42a5d73fc9b5ddc75cd1b4,2016-12-01,21:35:45 +0100,Kevin Cooney <kcooney@google.com>,kcooney@users.noreply.github.com,Kevin Cooney <kcooney@users.noreply.github.com>,Stefan Birkner <github@stefan-birkner.de>,Change generics on ExpectedException.expectCau...,M,src/main/java/org/junit/rules/ExpectedExceptio...,java,ExpectedException,/org/junit/rules
3,44cd8f80dbac11849a42a5d73fc9b5ddc75cd1b4,2016-12-01,21:35:45 +0100,Kevin Cooney <kcooney@google.com>,kcooney@users.noreply.github.com,Kevin Cooney <kcooney@users.noreply.github.com>,Stefan Birkner <github@stefan-birkner.de>,Change generics on ExpectedException.expectCau...,M,src/main/java/org/junit/rules/ExpectedExceptio...,java,ExpectedException$ExpectedExceptionStatement,/org/junit/rules
4,8baae6ecd6e719c8de58bac1a191de2badd78ed3,2016-12-08,19:16:32 +0100,Kevin Cooney <kcooney@google.com>,kcooney@users.noreply.github.com,Kevin Cooney <kcooney@users.noreply.github.com>,Kevin Cooney <kcooney@google.com>,Do not use @Override when implementing methods...,M,src/main/java/org/junit/internal/management/Fa...,java,FakeThreadMXBean,/org/junit/internal/management


In [3]:
classes.head()

,package,class,fileName,fullyQualifiedName,sourceFileName,javaVersion,md5,method,methodSignature,firstLine,lastLine,lines,cyclomaticComplexity,validClass,classVisibility,methodVisibility
0,/junit/extensions,ActiveTestSuite,/junit/extensions/ActiveTestSuite.class,junit.extensions.ActiveTestSuite,ActiveTestSuite.java,Java 5,2BDAADC0D257FF135888B3ABDFDEF7C7,<init>,void <init>(java.lang.String),25,26,2,1,True,public,public
1,/junit/extensions,ActiveTestSuite,/junit/extensions/ActiveTestSuite.class,junit.extensions.ActiveTestSuite,ActiveTestSuite.java,Java 5,2BDAADC0D257FF135888B3ABDFDEF7C7,<init>,void <init>(java.lang.Class),21,22,2,1,True,public,public
2,/junit/extensions,ActiveTestSuite,/junit/extensions/ActiveTestSuite.class,junit.extensions.ActiveTestSuite,ActiveTestSuite.java,Java 5,2BDAADC0D257FF135888B3ABDFDEF7C7,<init>,void <init>(),17,18,2,1,True,public,public
3,/junit/extensions,ActiveTestSuite,/junit/extensions/ActiveTestSuite.class,junit.extensions.ActiveTestSuite,ActiveTestSuite.java,Java 5,2BDAADC0D257FF135888B3ABDFDEF7C7,<init>,"void <init>(java.lang.Class,java.lang.String)",29,30,2,1,True,public,public
4,/junit/extensions,ActiveTestSuite,/junit/extensions/ActiveTestSuite.class,junit.extensions.ActiveTestSuite,ActiveTestSuite.java,Java 5,2BDAADC0D257FF135888B3ABDFDEF7C7,run,void run(junit.framework.TestResult),34,37,4,1,True,public,public


## git File Changes

In [38]:
class_change_count = history[history["fileType"] == "java"]
class_change_count = class_change_count[["file", "package"]]

# Removes classes without package
class_change_count = class_change_count[class_change_count["package"] != ""]
class_change_count = class_change_count[["file"]]
class_change_count["file"] = class_change_count["file"].str.replace(r"^src/","")
class_change_count["file"] = class_change_count["file"].str.replace("/",".")

# Clean up
class_change_count = class_change_count.value_counts()
class_change_count = class_change_count.reset_index(name="changes")
class_change_count = class_change_count.sort_values("file")
class_change_count = class_change_count.set_index("file")

class_change_count

,changes
file,
junit.extensions.ActiveTestSuite.java,4
junit.extensions.RepeatedTest.java,3
junit.extensions.TestDecorator.java,2
junit.extensions.TestSetup.java,4
junit.framework.Assert.java,2
...,...
org.junit.runner.notification.RunNotifier.java,44
org.junit.runner.notification.StoppedByUserException.java,2
org.junit.runners.AllTests.java,6


## Class Complexity

In [17]:
def getComplexity(lines, cyclomatic):
    total = 0
    
    if lines != "":
        total = lines * cyclomatic
    
    return total

In [32]:
classes["complexity"] = classes.apply(lambda x: getComplexity(x["lines"], x["cyclomaticComplexity"]), axis=1)

class_complexity = classes[["fullyQualifiedName", "complexity"]]
class_complexity = class_complexity[class_complexity["complexity"] != 0]
class_complexity = class_complexity.sort_values("fullyQualifiedName")
class_complexity = class_complexity.groupby(["fullyQualifiedName"]).sum()

class_complexity

,complexity
fullyQualifiedName,
junit.extensions.ActiveTestSuite,36.0
junit.extensions.ActiveTestSuite$1,9.0
junit.extensions.RepeatedTest,37.0
junit.extensions.TestDecorator,10.0
junit.extensions.TestSetup,7.0
...,...
org.junit.validator.AnnotationsValidator$AnnotatableValidator,69.0
org.junit.validator.AnnotationsValidator$ClassValidator,5.0
org.junit.validator.AnnotationsValidator$FieldValidator,5.0
